In [112]:
#NLP libraries
import findspark
findspark.init()

import re

import pyspark.sql.functions as psf
from pyspark.sql import *
from pyspark.sql.types import *

import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [113]:
# load data 
# keeping only the subtiles
df_text = spark.read.parquet('parquets/films2.parquet').select('tconst','subtitles')
df_ratings = spark.read.parquet('parquets/ratings.parquet')
df_basics = spark.read.parquet('parquets/basics.parquet')

In [114]:
# Joining with ratings to get best and worst movies 
df_tr = df_text.join(other=df_ratings.select('tconst','averageRating'), on='tconst', how='inner')
df_tr = df_tr.join(other=df_basics.select('tconst','genres'), on='tconst', how='inner')

In [115]:
df_tr = df_tr.select("tconst",'averageRating', 'subtitles',psf.explode_outer("genres"))

In [116]:
df_best = df_tr.filter(df_tr['averageRating'] >= 8)
df_worst = df_tr.filter(df_tr['averageRating'] <= 5.2)

# Detokinizing the text

In [117]:
from sacremoses import MosesDetokenizer
def subtitles_to_string(subtitles):
    result = ""
    for subtitle in subtitles:
        test = MosesDetokenizer().detokenize(subtitle)
        result += test + "\n"
    return result
udf_subtitles_to_string = psf.udf(subtitles_to_string, StringType())

In [118]:
df_best = df_best.withColumn("text",udf_subtitles_to_string("subtitles"))
df_best = df_best.drop('subtitles')

In [119]:
df_best.show()

+---------+-------------+---------+--------------------+
|   tconst|averageRating|      col|                text|
+---------+-------------+---------+--------------------+
|tt0032553|          8.5|   Comedy|At the end of the...|
|tt0032553|          8.5|    Drama|At the end of the...|
|tt0032553|          8.5|      War|At the end of the...|
|tt0032599|          8.0|   Comedy|Copy boy!
- Make ...|
|tt0032599|          8.0|    Drama|Copy boy!
- Make ...|
|tt0032599|          8.0|  Romance|Copy boy!
- Make ...|
|tt1280558|          8.2|    Crime|I am retiring tom...|
|tt1280558|          8.2|    Drama|I am retiring tom...|
|tt1280558|          8.2|  Mystery|I am retiring tom...|
|tt0038733|          8.1|   Comedy|This is the unive...|
|tt0038733|          8.1|    Drama|This is the unive...|
|tt0038733|          8.1|  Fantasy|This is the unive...|
|tt0063522|          8.0|    Drama|La la la la-la
La...|
|tt0063522|          8.0|   Horror|La la la la-la
La...|
|tt2488496|          8.0|   Act

In [120]:
df_worst = df_worst.withColumn("text",udf_subtitles_to_string("subtitles"))
df_worst = df_worst.drop('subtitles')

In [121]:
df_worst.show()

+---------+-------------+--------+--------------------+
|   tconst|averageRating|     col|                text|
+---------+-------------+--------+--------------------+
|tt0865554|          5.0|   Crime|What the fuck?
Te...|
|tt0865554|          5.0|   Drama|What the fuck?
Te...|
|tt0865554|          5.0|Thriller|What the fuck?
Te...|
|tt1781775|          4.5|  Horror|[ inaudible male ...|
|tt1781775|          4.5|Thriller|[ inaudible male ...|
|tt0775440|          4.3|  Action|Terminal A of Bos...|
|tt0775440|          4.3|   Drama|Terminal A of Bos...|
|tt0775440|          4.3|  Horror|Terminal A of Bos...|
|tt0962726|          4.7|  Comedy|[ Cheering]
[ Lab...|
|tt0962726|          4.7|   Drama|[ Cheering]
[ Lab...|
|tt0962726|          4.7|  Family|[ Cheering]
[ Lab...|
|tt0106400|          4.9|  Comedy|- I swear, if'n y...|
|tt0106400|          4.9|  Family|- I swear, if'n y...|
|tt0374536|          4.8|  Comedy|It's perfect.
Oh,...|
|tt0374536|          4.8| Fantasy|It's perfect.


# NLP modafucker

In [123]:
nlp = spacy.load('en')
nlp.remove_pipe('parser')
nlp.remove_pipe('tagger')

('tagger', <spacy.pipeline.Tagger at 0x1a3545cb70>)

In [43]:
genre = 'Comedy'

In [44]:
df_best_pd = df_best.filter(df_best['col'] == genre).toPandas()

In [45]:
df_best_pd.count()

tconst           80
averageRating    80
col              80
text             80
dtype: int64

In [46]:
texts_bests = df_best_pd['text'].values

In [47]:
# remove new lines 
texts_bests = [" ".join(t.split()) for t in texts_bests]

In [48]:
len(texts_bests)

80

In [49]:
texts_bests = [text.lower() for text in texts_bests]

### NLP pipe

In [50]:
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS

processed_docs = list()
for doc in nlp.pipe(texts_bests, n_threads=5, batch_size=10):

    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list and keep only words of length 3 or more.
    doc = [token for token in doc if token not in STOPWORDS and len(token) > 2]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)
    
docs = processed_docs ############### variable pour la suite

del processed_docs

In [51]:
# Add bigrams too
from gensim.models.phrases import Phrases

# Add bigrams to docs (only ones that appear 15 times or more).
bigram = Phrases(docs, min_count=4)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser 

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser 

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser 

In [52]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]
#MmCorpus.serialize("models/corpus.mm", corpus)

print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(corpus))

Number of unique tokens: 3504
Number of chunks: 80


In [53]:
seed = 42
# models
from gensim.models import LdaMulticore
params = {'passes': 10, 'random_state': seed}
base_models = dict()

model = LdaMulticore(corpus=corpus,
                     num_topics=3,
                     id2word=dictionary,
                     workers=6,
                     passes=params['passes'],
                     random_state=params['random_state'])

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

...

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\Martin\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:

In [54]:
model.show_topics(num_words=10)

[(0,
  '0.010*"mrs" + 0.007*"darling" + 0.007*"teacher" + 0.005*"senator" + 0.005*"fellow" + 0.005*"goodbye" + 0.004*"school" + 0.004*"new_york" + 0.004*"york" + 0.003*"george"'),
 (1,
  '0.009*"comrade" + 0.004*"dad" + 0.003*"school" + 0.003*"cook" + 0.003*"band" + 0.003*"plot" + 0.003*"chef" + 0.003*"david" + 0.003*"rat" + 0.003*"monsieur"'),
 (2,
  '0.008*"fuck" + 0.005*"dad" + 0.004*"uncle" + 0.003*"hell" + 0.003*"marriage" + 0.003*"shit" + 0.003*"village" + 0.003*"bitch" + 0.003*"gun" + 0.003*"jack"')]

In [55]:
model.show_topic(1,20)

[('comrade', 0.009286888),
 ('dad', 0.004397977),
 ('school', 0.0034351675),
 ('cook', 0.0032935133),
 ('band', 0.0031750435),
 ('plot', 0.0031522494),
 ('chef', 0.0029775489),
 ('david', 0.0028065632),
 ('rat', 0.0027887246),
 ('monsieur', 0.0027287847),
 ('damn', 0.0024193844),
 ('huh', 0.002367595),
 ('joy', 0.002352771),
 ('papa', 0.002298023),
 ('key', 0.002196306),
 ('report', 0.002134844),
 ('paris', 0.002132401),
 ('safe', 0.0021057385),
 ('darling', 0.0020449976),
 ('fish', 0.002036208)]

In [56]:
sorted(model[corpus[0]],key=lambda x:x[1],reverse=True)

[(2, 0.5816899), (0, 0.4053155), (1, 0.0129946135)]

In [57]:
# plot topics
data =  pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(data)

C:\Users\Martin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))
